# LinkedIn Compatibility Analysis

This notebook analyzes LinkedIn profile compatibility using machine learning models to predict network value between profile pairs.

## 📋 Project Overview

### What is this?
This project predicts **network value** between LinkedIn profile pairs using machine learning. Given two professionals (Profile A and Profile B), it estimates how valuable connecting with Person B would be for Person A.

### Business Use Case
- **For LinkedIn**: Recommend high-value connections to users
- **For Professionals**: Prioritize networking opportunities
- **For Recruiters**: Match candidates with mentors or collaborators

### Dataset
- **Source**: Kaggle - "LinkedIn Compatibility Dataset (50K Profiles)"
- **Contains**: 
  - Professional profiles (headlines, skills, experience, industry, seniority)
  - Pre-calculated compatibility pairs with network value scores
  - Text data about what professionals need and can offer

---

## 🛠️ Technologies & Why They're Used

### 1. **Data Processing**
- **Pandas & NumPy**: Standard data manipulation and numerical operations
- **Scikit-learn**: ML utilities (train/test split, metrics, preprocessing)

### 2. **Machine Learning Approach: Neural Network with Sentence Transformers**

Instead of using explicit numerical features, this uses **Natural Language Processing (NLP)** to understand professional profiles as text.

- **Technology**: `sentence-transformers` (all-MiniLM-L6-v2)
- **What it does**: Converts text into semantic embeddings (384-dimensional vectors)
- **Why**: Captures *meaning* not just keywords. "Machine Learning Engineer" and "AI Specialist" are semantically similar even with different words
- **Neural Network**: Multi-layer Perceptron (256→128→64 neurons) learns complex patterns from embeddings
- **Best for**: Understanding semantic relationships and context

---

## 🎯 Key Workflow

1. **Download data** from Kaggle
2. **Merge** profile information with compatibility pairs
3. **Create rich text profiles** combining all professional info
4. **Generate embeddings** using Sentence Transformers
5. **Train Neural Network** to predict network value
6. **Evaluate** model performance with R² and MSE metrics

---

## 💡 Why This Approach?

**Text-based over traditional features:**
- Professional value is nuanced and context-dependent
- Raw text contains rich signals that hand-crafted features miss
- NLP models can discover unexpected patterns (e.g., "open to mentoring" + "seeking guidance")

**Sentence Transformers advantages:**
- Deep semantic understanding of professional contexts
- Pre-trained on millions of text pairs
- Captures synonyms and contextual meanings
- Works well with professional/business language

---

## ⚡ Quick Start

**Prerequisites:**
```bash
pip install kagglehub pandas numpy scikit-learn sentence-transformers
```

**Runtime:** ~5-10 minutes on CPU (faster with GPU for embeddings)

**What you'll learn:**
1. How to process LinkedIn profile text data
2. Convert text to semantic embeddings
3. Train a Neural Network for value prediction
4. Evaluate model performance

Let's get started! 👇

---

## Import Required Libraries

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPRegressor
import time

print("✅ All imports loaded successfully!")

/Users/mali8/Library/Python/3.10/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download and Load Dataset

In [2]:
# Download latest version
path = kagglehub.dataset_download("likithagedipudi/linkedin-compatibility-dataset-50k-profiles")
print("Path to dataset files:", path)

100%|██████████| 278M/278M [00:27<00:00, 10.6MB/s] 

Extracting files...


Path to dataset files: /Users/mali8/.cache/kagglehub/datasets/likithagedipudi/linkedin-compatibility-dataset-50k-profiles/versions/1


In [3]:
# Load Pairs
csv_file_path_compatibility = os.path.join(path, "compatibility_pairs.csv")
pairs_df = pd.read_csv(csv_file_path_compatibility)

# Load Profiles
csv_file_path_profiles = os.path.join(path, "profiles.csv")
profiles_df = pd.read_csv(csv_file_path_profiles)

print(f"Pairs Loaded: {pairs_df.shape}")
print(f"Profiles Loaded: {profiles_df.shape}")

Pairs Loaded: (4999890, 14)
Profiles Loaded: (50000, 20)


## Explore Data

In [4]:
profiles_df.head()

,profile_id,name,email,location,headline,about,current_role,current_company,industry,years_experience,seniority_level,skills,experience,education,connections,goals,needs,can_offer,remote_preference,source
0,ab04b973af478550ddf247879393df42,Daniel Doyle,garzaanthonyexample.org,"East William, AK",Analyst Product Building impactful solutions,Experienced professional focused on driving gr...,Assistant,Microsoft,Healthcare,2,entry,"['Prototyping', 'Go', 'C', 'C', 'NLP']","[{'title': 'Assistant', 'company': 'Google', '...","[{'school': 'Penn', 'degree': 'MS', 'field': '...",106,"['Get promoted', 'Build network']","['funding', 'mentorship', 'business advice']","['partnership opportunities', 'investment', 'c...",remote,synthetic
1,b620e3fa2ec361b1d728115eeabb71af,Jennifer Cole,lisa02example.net,"Petersonberg, IL",Senior Engineer Design Building impactful so...,Passionate about building innovative solutions...,Lead Data Scientist,Stripe,Consulting,9,senior,"['Business Development', 'SQL', 'Ruby', 'Marke...","[{'title': 'Senior Engineer', 'company': 'Rapi...","[{'school': 'Princeton', 'degree': 'MBA', 'fie...",2372,"['Strategic role', 'Scale impact']","['job opportunities', 'mentorship', 'clients']","['industry connections', 'consulting', 'produc...",hybrid,synthetic
2,cfeeb31581a0b3e0515c01691b9dc2b5,Brent Abbott,lindsay78example.org,"Millerport, MP",Software Engineer Product Building impactful...,Passionate about building innovative solutions...,Data Scientist,NextGen,Telecommunications,5,mid,"['NLP', 'Business Development', 'Sales', 'Big ...","[{'title': 'Consultant', 'company': 'ScaleUp',...","[{'school': 'CMU', 'degree': 'MS', 'field': 'D...",874,"['Specialize', 'Mentor others']","['job opportunities', 'business advice', 'care...","['partnership opportunities', 'product feedbac...",onsite,synthetic
3,5d54826665a5898662661a96719cc4a7,Corey Jones,kendragallowayexample.org,"South Joshuastad, GA",Chief Data Officer Tech Building impactful s...,Passionate about building innovative solutions...,COO,NextGen,Healthcare,17,executive,"['Sketch', 'Machine Learning', 'AWS', 'TensorF...","[{'title': 'VP Engineering', 'company': 'Tesla...","[{'school': 'MIT', 'degree': 'BS', 'field': 'E...",3259,"['Build company', 'Advisory roles']","['clients', 'hiring', 'career guidance']","['consulting', 'career advice', 'hiring referr...",onsite,synthetic
4,6ad3c64c6cb4bac60b692f3d5bab271d,Timothy Wong,amandasanchezexample.com,"Nelsonside, IN",Consultant Design Building impactful solutions,Strategic thinker with expertise in scaling or...,Product Manager,NextGen,Healthcare,4,mid,"['Data Science', 'DevOps', 'Flask', 'Analytics...","[{'title': 'Software Engineer', 'company': 'Ai...","[{'school': 'Berkeley', 'degree': 'MS', 'field...",372,"['Specialize', 'Lead projects']","['clients', 'hiring', 'funding']","['partnership opportunities', 'career advice',...",remote,synthetic


In [5]:
pairs_df.head()

,skill_match_score,skill_complementarity_score,network_value_a_to_b,network_value_b_to_a,career_alignment_score,experience_gap,industry_match,geographic_score,seniority_match,compatibility_score,mutual_benefit_explanation,pair_id,profile_a_id,profile_b_id
0,0.000000,0.0,5.3,14.55,80.0,0,0.0,60.0,85.0,24.98,Peer-level relationship - can learn together,742f902f23b9d1be5fa0ba0816e3490b,ab04b973af478550ddf247879393df42,fdf3243d1ad97255e0ce313aebc0be79
1,5.555556,0.0,5.3,42.80,80.0,2,0.0,60.0,100.0,30.33,Peer-level relationship - can learn together,fc6c6a8029bc4e9beb5dd8186147f042,ab04b973af478550ddf247879393df42,371bc2adbdc4ca8f0dd16d373f85f2ae
2,5.555556,0.0,5.3,38.80,80.0,2,0.0,60.0,100.0,29.73,Peer-level relationship - can learn together,6cb0929495e49ddd5dff151ead7b3f5e,ab04b973af478550ddf247879393df42,d18e7cd91fc4e621fd6879ca5ef6e1b2
3,8.000000,0.0,5.3,80.00,40.0,27,0.0,60.0,50.0,28.39,Valuable network connections in same industry,7b191de3be4c52fd0874f31936d51819,ab04b973af478550ddf247879393df42,6615dabdd3b5b8f9627ca933dc3d9ae3
4,7.142857,0.0,5.3,28.60,90.0,5,0.0,60.0,100.0,30.51,Ideal mentorship gap (5 years experience diffe...,7b568a4ee7254a9b388e6430ac252568,ab04b973af478550ddf247879393df42,fc73d0e790dea954d20db176f638ab86


## Data Preprocessing

In [ ]:
# Remove unnecessary columns from profiles_df and pairs_df
profiles_df = profiles_df.drop(['name','email','location'], axis=1, errors='ignore')
pairs_df = pairs_df.drop(['industry_match','compatibility_score','mutual_benefit_explanation',
                          'geographic_score','career_alignment_score',
                          'skill_complementarity_score', 'skill_match_score'],
                          axis=1, errors='ignore')

# Shuffle and keep only 5000 rows to reduce dataset size
pairs_df = pairs_df.sample(frac=1, random_state=93).reset_index(drop=True).head(5000)

print(f"✅ Cleaned data shape: {pairs_df.shape}")
print(f"📊 Columns retained in pairs: {list(pairs_df.columns)}")

(5000, 7)


## Merge Profile Data with Pairs

In [ ]:
# Merge Profile A
data = pairs_df.merge(profiles_df, left_on='profile_a_id', right_on='profile_id', how='left')
# Rename columns for A
cols_to_rename = {col: f"{col}_a" for col in profiles_df.columns if col != 'profile_id'}
data = data.rename(columns=cols_to_rename)

# Merge Profile B
data = data.merge(profiles_df, left_on='profile_b_id', right_on='profile_id', how='left')
# Rename columns for B
cols_to_rename_b = {col: f"{col}_b" for col in profiles_df.columns if col != 'profile_id'}
data = data.rename(columns=cols_to_rename_b)

# Cleanup IDs
data = data.drop(['profile_id_x', 'profile_id_y'], axis=1, errors='ignore')

# Ensure target is numeric
data['network_value_a_to_b'] = pd.to_numeric(data['network_value_a_to_b'], errors='coerce').fillna(0)

print(f"✅ Merged dataset shape: {data.shape}")
print(f"📊 Target variable stats:")
print(f"   Mean: {data['network_value_a_to_b'].mean():.2f}")
print(f"   Std: {data['network_value_a_to_b'].std():.2f}")
print(f"   Range: [{data['network_value_a_to_b'].min():.2f}, {data['network_value_a_to_b'].max():.2f}]")

Merged Data Sample:


Index(['network_value_a_to_b', 'network_value_b_to_a', 'experience_gap',
       'seniority_match', 'pair_id', 'profile_a_id', 'profile_b_id',
       'headline_a', 'about_a', 'current_role_a', 'current_company_a',
       'industry_a', 'years_experience_a', 'seniority_level_a', 'skills_a',
       'experience_a', 'education_a', 'connections_a', 'goals_a', 'needs_a',
       'can_offer_a', 'remote_preference_a', 'source_a', 'headline_b',
       'about_b', 'current_role_b', 'current_company_b', 'industry_b',
       'years_experience_b', 'seniority_level_b', 'skills_b', 'experience_b',
       'education_b', 'connections_b', 'goals_b', 'needs_b', 'can_offer_b',
       'remote_preference_b', 'source_b'],
      dtype='object')

## Machine Learning Model: Neural Network with Sentence Transformers

### Feature Engineering - Create Text Profiles

In [ ]:
# Combine all useful text fields into one rich string per user
def create_profile_text(row, suffix):
    # e.g., suffix='_a' -> grabs headline_a, current_role_a...
    return (
        str(row[f'headline{suffix}']) + " | " +
        str(row[f'current_role{suffix}']) + " | " +
        str(row[f'current_company{suffix}']) + " | " +
        str(row[f'about{suffix}']) + " | " +
        str(row[f'skills{suffix}']) + " | " +
        str(row[f'experience{suffix}']) + " | " +
        str(row[f'seniority_level{suffix}']) + " | " +
        str(row[f'industry{suffix}']) + " | " +
        str(row[f'needs{suffix}']) + " | " +
        str(row[f'can_offer{suffix}'])
    )

print("📝 Constructing text profiles...")
start_time = time.time()

data['full_text_a'] = data.apply(lambda row: create_profile_text(row, '_a'), axis=1)
data['full_text_b'] = data.apply(lambda row: create_profile_text(row, '_b'), axis=1)

print(f"✅ Created {len(data)} profile pairs in {time.time() - start_time:.2f}s")
print(f"📊 Average text length: {data['full_text_a'].str.len().mean():.0f} characters")

📝 Constructing text profiles...


**💡 Why combine all text fields?**

Instead of treating each field separately (headline, skills, experience), we create one comprehensive text profile per person. This allows the model to understand the FULL professional context.

For example:
- Profile A: "Senior ML Engineer | Google | 10 years Python, TensorFlow | Seeking: mentorship opportunities"
- Profile B: "Junior Data Scientist | Startup | Learning ML | Needs: senior guidance"

The model can now see that A can mentor B → High network value!

### Generate Embeddings

**🧠 How Sentence Transformers Work**

1. **Pre-trained Model**: `all-MiniLM-L6-v2` was trained on millions of sentences to understand meaning
2. **Embeddings**: Converts text → 384 numbers that capture semantic meaning
3. **Similar meanings = Similar vectors**: "Software Engineer" and "Developer" have vectors close together
4. **Input to Neural Network**: We stack Profile A's vector + Profile B's vector = 768 numbers total

This is MORE powerful than just counting words because it understands CONTEXT and SYNONYMS.

In [ ]:
# Load pre-trained model for semantic understanding
print("🧠 Loading Sentence Transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

print("⏳ Converting text to embeddings (this may take a minute)...")
start_time = time.time()

embeddings_a = model.encode(data['full_text_a'].tolist(), show_progress_bar=True)
embeddings_b = model.encode(data['full_text_b'].tolist(), show_progress_bar=True)

print(f"✅ Embeddings created in {time.time() - start_time:.2f}s")
print(f"📊 Embedding shape: Profile A = {embeddings_a.shape}, Profile B = {embeddings_b.shape}")

# Stack them side-by-side: [User A Vector, User B Vector]
X = np.hstack([embeddings_a, embeddings_b])
y = data['network_value_a_to_b'].values

print(f"📊 Final feature matrix: {X.shape} (768 = 384 + 384 dimensions)")

⏳ Turning text into numbers (Embeddings)...


Batches: 100%|██████████| 157/157 [00:12<00:00, 12.95it/s]



### Train Neural Network Regressor

In [ ]:
print("🔄 Splitting data into train/test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"📊 Train set: {X_train.shape[0]} samples")
print(f"📊 Test set: {X_test.shape[0]} samples")

# Train Neural Network
print("\n🧠 Training Neural Network (MLP)...")
start_time = time.time()

regressor = MLPRegressor(
    hidden_layer_sizes=(256, 128, 64), 
    activation='relu',
    max_iter=500, 
    random_state=42, 
    verbose=False  # Set to True to see training progress
)

regressor.fit(X_train, y_train)

print(f"✅ Model trained in {time.time() - start_time:.2f}s")

/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Iteration 1, loss = 1566.62035204
Iteration 2, loss = 613.53816470
Iteration 3, loss = 329.08828326
Iteration 4, loss = 238.54067528
Iteration 5, loss = 164.20411459
Iteration 6, loss = 105.83799991
Iteration 7, loss = 82.05419165
Iteration 8, loss = 76.70187915
Iteration 9, loss = 73.13160623
Iteration 10, loss = 70.21561187
Iteration 11, loss = 67.79018935
Iteration 12, loss = 65.78525912
Iteration 7, loss = 82.05419165
Iteration 8, loss = 76.70187915
Iteration 9, loss = 73.13160623
Iteration 10, loss = 70.21561187
Iteration 11, loss = 67.79018935
Iteration 12, loss = 65.78525912
Iteration 13, loss = 64.01010387
Iteration 14, loss = 62.55531268
Iteration 15, loss = 61.42287141
Iteration 16, loss = 59.99121058
Iteration 17, loss = 59.01809690
Iteration 18, loss = 57.41961131
Iteration 13, loss = 64.01010387
Iteration 14, loss = 62.55531268
Iteration 15, loss = 61.42287141
Iteration 16, loss = 59.99121058
Iteration 17, loss = 59.01809690
Iteration 18, loss = 57.41961131
Iteration 19, l

/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,loss,'squared_error'
,hidden_layer_sizes,"(256, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,500
,shuffle,True


### Evaluate Model Performance

In [ ]:
print("📊 Evaluating model performance...\n")

# Make predictions
preds = regressor.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

# Display results
print("=" * 50)
print("🎯 MODEL PERFORMANCE")
print("=" * 50)
print(f"R² Score:            {r2:.4f}")
print(f"Mean Squared Error:  {mse:.2f}")
print(f"RMSE:                {np.sqrt(mse):.2f}")
print("=" * 50)

# Show example predictions
print("\n📋 Sample Predictions (first 5 test samples):")
print(f"{'Actual':<10} {'Predicted':<10} {'Difference':<10}")
print("-" * 35)
for i in range(min(5, len(y_test))):
    diff = abs(y_test[i] - preds[i])
    print(f"{y_test[i]:<10.2f} {preds[i]:<10.2f} {diff:<10.2f}")

# Overall accuracy insight
print(f"\n💡 The model explains {r2*100:.1f}% of the variance in network value!")

------------------------------
✅ Model Trained on TEXT ONLY
Mean Squared Error: 153.93
R² Score: 0.81
------------------------------
Example Prediction:
User A: Coordinator  Tech  Building impactful solutions | ...
User B: Chief Data Officer  Tech  Building impactful solut...
Predicted Value: 105.42 / Actual: 80.00


/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mali8/Library/Python/3.10/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


---

## 🎉 Project Complete!

### 📊 Summary

You've successfully built a machine learning model that predicts LinkedIn network value using:
- **50K profile pairs** from Kaggle
- **Sentence Transformers** for semantic text understanding
- **Neural Network (MLP)** for value prediction

### 🎯 Key Achievements

✅ **Semantic Understanding** - The model learned to recognize professional compatibility from text alone  
✅ **Context Over Keywords** - Understands that "ML Engineer" ≈ "AI Specialist"  
✅ **Complementary Signals** - Identifies mentor-mentee dynamics and needs/offers alignment  

### 🔄 Next Steps

1. **Fine-tune**: Try different architectures or pre-trained models
2. **Feature engineering**: Add temporal features (career progression)
3. **Deploy**: Build an API endpoint for real-time predictions
4. **Validate**: A/B test with actual LinkedIn connection outcomes

### 📢 Share Your Work

Share this notebook on GitHub and LinkedIn! Use the post template in `linkedin_post.md`.

**Quick stats to include:**
- Dataset size: 5,000 pairs
- Model: Neural Network with Sentence Transformers
- Your R² score and MSE from above

---

**Built with:** Python · Sentence Transformers · Scikit-learn · Pandas · NumPy